In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\do04\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\do04\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('McD_Rev.csv',encoding='cp1252')
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [3]:
df['rating'] = df['rating'].str.replace('star', '')
df['rating'] = df['rating'].str.replace('s', '')

In [4]:
df

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5


In [5]:
df.dtypes

df['rating'] = df['rating'].astype(int)

In [6]:
def map_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating <= 2:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to create a new column 'sentiment'
df['sentiment'] = df['rating'].apply(map_sentiment)



def map_nos(charac):
    if charac == 'positive':
        return 0
    elif charac == 'negative':
        return 2
    elif charac == 'neutral':
        return 1


df['sentiment_nos'] = df['sentiment'].apply(map_nos)

df


,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating,sentiment,sentiment_nos
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1,negative,2
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4,positive,0
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1,negative,2
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5,positive,0
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1,negative,2
...,...,...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1,negative,2
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5,positive,0
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4,positive,0
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5,positive,0


In [7]:
columns_to_drop = ['reviewer_id','store_name','category','store_address','latitude ','longitude','rating_count','review_time','rating','sentiment']

In [8]:
df=df.drop(columns=columns_to_drop)

In [9]:
df

,review,sentiment_nos
0,Why does it look like someone spit on my food?...,2
1,It'd McDonalds. It is what it is as far as the...,0
2,Made a mobile order got to the speaker and che...,2
3,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,0
4,"I repeat my order 3 times in the drive thru, a...",2
...,...,...
33391,They treated me very badly.,2
33392,The service is very good,0
33393,To remove hunger is enough,0
33394,"It's good, but lately it has become very expen...",0


In [10]:
df.isnull().sum()

review           0
sentiment_nos    0
dtype: int64

In [11]:
#Preprocessing text


In [12]:
def clean_text(text):
    # Lowercasing
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Removing punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [token.translate(table) for token in tokens]
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Joining tokens back into a string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

In [13]:
df['review'] = df['review'].apply(clean_text)
df

,review,sentiment_nos
0,look like someon spit food normal transact e...,2
1,mcdonald far food atmospher go staff make di...,0
2,made mobil order got speaker check line move ...,2
3,mc crispi chicken sandwich ï¿½ï¿½ï¿½ï¿½ï¿½ï¿½...,0
4,repeat order 3 time drive thru still manag me...,2
...,...,...
33391,treat badli,2
33392,servic good,0
33393,remov hunger enough,0
33394,good late becom expens,0


In [14]:
X=df['review']
y=df['sentiment_nos']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [16]:
models={'MultinomialNB':MultinomialNB(),'SVM':SVC(),'LogisticRegression':LogisticRegression(),'KNeighborsClassifier':KNeighborsClassifier()}

model=[]
acc_l=[]

for key, value in models.items():
    nb = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   (key, value)
                   ])
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    acc=accuracy_score(y_test,y_pred)


    model.append(key)

    acc_l.append(acc)


final={}

for key, value in zip(model, acc_l):
    final[key] = value

print(final)

max_accuracy = max(final.values())

key_best = [key for key, value in final.items() if value == max_accuracy]
a=key_best[0]
b=str(a)


selected_model = models.get(b)

{'MultinomialNB': 0.7932634730538922, 'SVM': 0.8261976047904191, 'LogisticRegression': 0.8115269461077844, 'KNeighborsClassifier': 0.6053892215568862}


In [19]:
new_input = ["The food taste is awesome"]
l=[]
for i in new_input:
    result = clean_text(i)
    l.append(result)
    n = Pipeline([('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('selected_model', selected_model)])
    n.fit(X_train, y_train)
    prediction = n.predict(l)
    if prediction==0:
        print("Review is positive")
    elif prediction==2:
        print("Review is negative")
    elif prediction==1:
        print("Review is neutral")

Review is positive
